In [ ]:
# !pip install pyspark

# Índice
*Notebook contendo apenas algumas funções basicas*

1. [Imports](#Import)
2. [Usando dropna](#dropna)
3. [Dropando duplicados](#dropDuplicates)
4. [Função Sample](#sample)
5. [Filtrando com filter ou where](#filter_where)
6. [Joins](#Join)
7. [Agregação(agg)](#agg)
8. [Extras](#Extras)

## Import

In [75]:
# Importação das bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, concat, round, substring

In [3]:
# Inicializar uma sessão do Spark e impedindo avisos desnecessários
spark = SparkSession.builder \
    .appName("NomeDaApp") \
    .config("spark.logConf", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

## dropna

In [11]:
# Exemplo de DataFrame com 10 registros e 4 campos (incluindo valores nulos/vazios)
data = [
    ("Alice", 34, "São Paulo", "Brasil"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Alice", None, "São Paulo", "Brasil"),  # Idade nula
    ("Carla", 29, "", "Portugal"),          # Cidade vazia
    ("David", 30, "Berlim", None),          # País nulo
    ("Eva", 25, "Paris", "França"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Frank", 40, "Tóquio", "Japão"),
    ("Grace", 35, None, "Reino Unido"),     # Cidade nula
    ("Hank", 50, "Sydney", "Austrália")
]

df = spark.createDataFrame(data, ["Nome", "Idade", "Cidade", "País"])

# Remover linhas com valores nulos ou vazios e salvar na mesma variável `df`
df = df.na.drop()  # Remove linhas com qualquer valor nulo ou vazio

# Mostrar o resultado
df.show()

+-----+-----+---------+---------+
| Nome|Idade|   Cidade|     País|
+-----+-----+---------+---------+
|Alice|   34|São Paulo|   Brasil|
|  Bob|   45|Nova York|      EUA|
|Carla|   29|         | Portugal|
|  Eva|   25|    Paris|   França|
|  Bob|   45|Nova York|      EUA|
|Frank|   40|   Tóquio|    Japão|
| Hank|   50|   Sydney|Austrália|
+-----+-----+---------+---------+



## dropDuplicates

In [10]:
# Exemplo de DataFrame com 10 registros e 4 campos
data = [
    ("Alice", 34, "São Paulo", "Brasil"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Alice", 34, "São Paulo", "Brasil"),
    ("Carla", 29, "Lisboa", "Portugal"),
    ("David", 30, "Berlim", "Alemanha"),
    ("Eva", 25, "Paris", "França"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Frank", 40, "Tóquio", "Japão"),
    ("Grace", 35, "Londres", "Reino Unido"),
    ("Hank", 50, "Sydney", "Austrália")
]

df = spark.createDataFrame(data, ["Nome", "Idade", "Cidade", "País"])

# Remover linhas duplicadas e salvar na mesma variável `df`
df = df.dropDuplicates()

# Mostrar o resultado
df.show()

+-----+-----+---------+-----------+
| Nome|Idade|   Cidade|       País|
+-----+-----+---------+-----------+
|Alice|   34|São Paulo|     Brasil|
|  Bob|   45|Nova York|        EUA|
|David|   30|   Berlim|   Alemanha|
|Carla|   29|   Lisboa|   Portugal|
|  Eva|   25|    Paris|     França|
|Frank|   40|   Tóquio|      Japão|
|Grace|   35|  Londres|Reino Unido|
| Hank|   50|   Sydney|  Austrália|
+-----+-----+---------+-----------+



## sample

In [32]:
# Dados de exemplo
data = [
    ("Alice", 25, "São Paulo"),
    ("Bob", 30, "Nova York"),
    ("Carla", 22, "Lisboa"),
    ("David", 35, "Berlim"),
    ("Eva", 28, "Paris"),
    ("Frank", 40, "Tóquio"),
    ("Grace", 33, "Londres"),
    ("Hank", 50, "Sydney"),
    ("Ivy", 27, "Toronto"),
    ("Jack", 29, "Cidade do México")
]

# Criar DataFrame
df = spark.createDataFrame(data, ["Nome", "Idade", "Cidade"])

# Aplicar sample para retornar uma amostra aleatória de 30% dos registros
df = df.sample(fraction=0.3, seed=None)

# Exibir a amostra
df.show()

+-----+-----+-------+
| Nome|Idade| Cidade|
+-----+-----+-------+
|  Eva|   28|  Paris|
|Frank|   40| Tóquio|
|Grace|   33|Londres|
+-----+-----+-------+



## filter_where

In [37]:
# Dados de exemplo
data = [
    ("Alice", 25, "São Paulo"),
    ("Bob", 30, "Nova York"),
    ("Carla", 22, "Lisboa"),
    ("David", 35, "Berlim"),
    ("Eva", 28, "Paris"),
    ("Frank", 40, "Tóquio"),
    ("Grace", 33, "Londres"),
    ("Hank", 50, "Sydney"),
    ("Ivy", 27, "Toronto"),
    ("Jack", 29, "Cidade do México")
]

# Criar DataFrame
df = spark.createDataFrame(data, ["Nome", "Idade", "Cidade"])

# Filtra linhas com filter onde a coluna "Idade" é maior que 30
print("Uso do filter")
df.filter(df["Idade"] > 30).show()

print("Uso do where")
df.where(df["Idade"] > 30).show()

Uso do filter
+-----+-----+-------+
| Nome|Idade| Cidade|
+-----+-----+-------+
|David|   35| Berlim|
|Frank|   40| Tóquio|
|Grace|   33|Londres|
| Hank|   50| Sydney|
+-----+-----+-------+

Uso do where
+-----+-----+-------+
| Nome|Idade| Cidade|
+-----+-----+-------+
|David|   35| Berlim|
|Frank|   40| Tóquio|
|Grace|   33|Londres|
| Hank|   50| Sydney|
+-----+-----+-------+



## Join

In [44]:
# Criando dois dataframes para que possam ser ligados pelo id da cidade
data_pessoas = [
    (1, "Alice", 1),
    (2, "Bob", 2),
    (3, "Carla", 3),
    (4, "David", 5)   
]
df_pessoas = spark.createDataFrame(data_pessoas, ["id_pessoa", "nome", "id_cidade"])
df_pessoas.show()

data_cidades = [
    (1, "São Paulo"),
    (2, "Nova York"),
    (3, "Lisboa"),
    (4, "Berlim")
]
df_cidades = spark.createDataFrame(data_cidades, ["id_cidade", "cidade"])
df_cidades.show()

# Realizando join
df_join = df_pessoas.join(df_cidades, df_pessoas.id_cidade == df_cidades.id_cidade, how="inner")
# Retornando resultado com select
df_join.select("id_pessoa", "nome", "cidade").show()

+---------+-----+---------+
|id_pessoa| nome|id_cidade|
+---------+-----+---------+
|        1|Alice|        1|
|        2|  Bob|        2|
|        3|Carla|        3|
|        4|David|        5|
+---------+-----+---------+

+---------+---------+
|id_cidade|   cidade|
+---------+---------+
|        1|São Paulo|
|        2|Nova York|
|        3|   Lisboa|
|        4|   Berlim|
+---------+---------+

+---------+-----+---------+
|id_pessoa| nome|   cidade|
+---------+-----+---------+
|        1|Alice|São Paulo|
|        2|  Bob|Nova York|
|        3|Carla|   Lisboa|
+---------+-----+---------+



##

## agg

In [67]:
# Dados de exemplo
data = [
    ("2024-11-26", 654.0, 38.9),
    ("2024-12-03", 602.1, 40.19),
    ("2024-12-18", 649.5, 42.80),
    ("2025-01-26", 636.9, 42.13)
]

# Criar DataFrame
df = spark.createDataFrame(data, ["data_abastecimento", "trip", "gasto_gasolina"])

# Usar agg para calcular o gasto de combustivel e criar uma nova coluna
df = df.withColumn("consumo_total", round(col("trip") / col("gasto_gasolina"), 2))

# Exibir o resultado
df.show()

+------------------+-----+--------------+-------------+
|data_abastecimento| trip|gasto_gasolina|consumo_total|
+------------------+-----+--------------+-------------+
|        2024-11-26|654.0|          38.9|        16.81|
|        2024-12-03|602.1|         40.19|        14.98|
|        2024-12-18|649.5|          42.8|        15.18|
|        2025-01-26|636.9|         42.13|        15.12|
+------------------+-----+--------------+-------------+



---

## Extras

### Caracteres Especiais

In [15]:
#### Função para checar caracteres especiais e retornar onde se encontram(util em dataframes grandes)
import pyspark.sql.functions as F
from pyspark.sql.functions import col,lit
from pyspark.sql.types import ArrayType, StringType, StructType, StructField, TimestampType
from pyspark.sql.functions import collect_list, concat_ws

def search_dataframe(df, tabela):

    intermediate_dfs = []

    for coluna in df.columns:

        processing_df = df.filter(F.lower(df[coluna]).rlike('[^a-z0-9\s!`´"#$%&\'()*+,-./\]\[|:;<=>?@\[\\áàâäãéèêëíìîïóòôöõúùûüñç]'))
        processing_df = processing_df.withColumn('COLUMN_ERROR', lit(coluna))\
            .withColumn('TABELA', lit(tabela))
        intermediate_dfs.append(processing_df)

    # Combine todos os DataFrames intermediários em um único DataFrame
    result_df = intermediate_dfs[0]
    for i in range(1, len(intermediate_dfs)):
        result_df = result_df.union(intermediate_dfs[i])

    colunas_referencia = df.columns
    # Exiba o DataFrame result_df
    # result_df = result_df.groupBy(df.columns,"TABELA"]).agg(concat_ws(', ',collect_list("COLUMN_ERROR")).alias('COLUMN_ERROR'))
    result_df = result_df.groupBy(*[col(coluna) for coluna in colunas_referencia] + [col("TABELA")]).agg(concat_ws(', ',collect_list("COLUMN_ERROR")).alias('COLUMN_ERROR'))

    # Retonando dataframe criado dataframe usado + coluna da tabela + coluna(s) com erros
    return result_df


In [16]:
data = [
    ("Alice☺", 25, "São Paulo", "Brasil", "Rua A"),
    ("Bob", 30, "Nova York♠", "EUA", "Rua B"),
    ("Carla", 22, "Lisboa", "Portugal✓", "Rua C"),
    ("David", 35, "Berlim∞", "Alemanha", "Rua D"),
    ("Eva", 28, "Paris", "França", "Rua E"),
    ("Frank", 40, "Tóquio☺", "Japão", "Rua F"),
    ("Grace", 33, "Londres♠", "Reino Unido", "Rua G"),
    ("Hank", 50, "Sydney", "Austrália✓", "Rua H"),
    ("Ivy", 27, "Toronto", "Canadá", "Rua I"),
    ("Jack®", 29, "Cidade do México", "México", "Rua J")
]

# Nomes das colunas
colunas = ["Nome", "Idade", "Cidade", "País", "Rua"]

# Criar DataFrame
df = spark.createDataFrame(data, colunas)

tabela = 'tabela_teste'
resultado = search_dataframe(df,tabela)
if resultado.isEmpty():
    print(f'Sem dados invalidos na tabela {tabela}')
else:
    resultado.show()

+------+-----+----------------+-----------+-----+------------+------------+
|  Nome|Idade|          Cidade|       País|  Rua|      TABELA|COLUMN_ERROR|
+------+-----+----------------+-----------+-----+------------+------------+
|Alice☺|   25|       São Paulo|     Brasil|Rua A|tabela_teste|        Nome|
| Jack®|   29|Cidade do México|     México|Rua J|tabela_teste|        Nome|
|   Bob|   30|      Nova York♠|        EUA|Rua B|tabela_teste|      Cidade|
| David|   35|         Berlim∞|   Alemanha|Rua D|tabela_teste|      Cidade|
| Frank|   40|         Tóquio☺|      Japão|Rua F|tabela_teste|      Cidade|
| Grace|   33|        Londres♠|Reino Unido|Rua G|tabela_teste|      Cidade|
| Carla|   22|          Lisboa|  Portugal✓|Rua C|tabela_teste|        País|
|  Hank|   50|          Sydney| Austrália✓|Rua H|tabela_teste|        País|
+------+-----+----------------+-----------+-----+------------+------------+



### Calculo_CNPJ
*Sabemos que os digitos verificadores do CNPJ, são na verdade um calculo dos numeros anteriores, algumas bases antigas usavam essa estrategia para reduzir custos de armazenamento, deixando que funções proprias calculassem exibido na consulta com os digitos verificadores*

In [90]:
data = [
    ("Empresa A", "123456780001", "00"),
    ("Empresa B", "987654320001", "12"),
    ("Empresa C", "456789120001", "00"),
    ("Empresa D", "321654980001", "34"),
    ("Empresa E", "654123780001", "00"),
    ("Empresa F", "789321450001", "00"),
    ("Empresa G", "852963740001", "00"),
    ("Empresa H", "963852140001", "78"),
    ("Empresa I", "741852960001", "00"),
    ("Empresa J", "258147360001", "90")
]

# Criar DataFrame com a ordem desejada
df = spark.createDataFrame(data, ["nome_empresa", "cnpj", "digito_verificador"])

tabela="empresas"
df.write.mode("overwrite").saveAsTable(tabela)
spark.sql("SELECT * FROM empresas").show()

+------------+------------+------------------+
|nome_empresa|        cnpj|digito_verificador|
+------------+------------+------------------+
|   Empresa A|123456780001|                00|
|   Empresa F|789321450001|                00|
|   Empresa I|741852960001|                00|
|   Empresa J|258147360001|                90|
|   Empresa G|852963740001|                00|
|   Empresa H|963852140001|                78|
|   Empresa D|321654980001|                34|
|   Empresa E|654123780001|                00|
|   Empresa B|987654320001|                12|
|   Empresa C|456789120001|                00|
+------------+------------+------------------+



In [104]:
# Calculando o primeiro DV
# TODO: Verificar calculo incorreto(script desatualizado)
try:
    df = df.withColumn("DV1", when((col("digito_verificador") == "00"), 
                        ((substring(col("cnpj"), 1, 1).cast('int') * 6) +
                        (substring(col("cnpj"), 2, 1).cast('int') * 7) +
                        (substring(col("cnpj"), 3, 1).cast('int') * 8) +
                        (substring(col("cnpj"), 4, 1).cast('int') * 9) +
                        (substring(col("cnpj"), 5, 1).cast('int') * 2) +
                        (substring(col("cnpj"), 6, 1).cast('int') * 3) +
                        (substring(col("cnpj"), 7, 1).cast('int') * 4) +
                        (substring(col("cnpj"), 8, 1).cast('int') * 5) +
                        (substring(col("cnpj"), 9, 1).cast('int') * 6) +
                        (substring(col("cnpj"), 10, 1).cast('int') * 7) +
                        (substring(col("cnpj"), 11, 1).cast('int') * 8) +
                        (substring(col("cnpj"), 12, 1).cast('int') * 9)) % 11).otherwise(''))
    df = df.withColumn('DV1', when(df.DV1 >= '10', '0').otherwise(col('DV1')))
except Exception as e:
    print(str(e))

# Calculando o segundo DV
try:
    df = df.withColumn("DV2", when((col("digito_verificador") == "00"), 
                        ((substring(col("cnpj"), 1, 1).cast('int') * 5) +
                        (substring(col("cnpj"), 2, 1).cast('int') * 6) +
                        (substring(col("cnpj"), 3, 1).cast('int') * 7) +
                        (substring(col("cnpj"), 4, 1).cast('int') * 8) +
                        (substring(col("cnpj"), 5, 1).cast('int') * 9) +
                        (substring(col("cnpj"), 6, 1).cast('int') * 2) +
                        (substring(col("cnpj"), 7, 1).cast('int') * 3) +
                        (substring(col("cnpj"), 8, 1).cast('int') * 4) +
                        (substring(col("cnpj"), 9, 1).cast('int') * 5) +
                        (substring(col("cnpj"), 10, 1).cast('int') * 6) +
                        (substring(col("cnpj"), 11, 1).cast('int') * 7) +
                        (substring(col("cnpj"), 12, 1).cast('int') * 8) +
                        (col('DV1').cast('int') * 9)) % 11).otherwise(''))
    df = df.withColumn('DV2', when(df.DV2 >= '10', '0').otherwise(col('DV2')))
except Exception as e:
    print(str(e))

# df.show()

In [96]:
df.show()

+------------+------------+------------------+---+---+
|nome_empresa|        cnpj|digito_verificador|DV1|DV2|
+------------+------------+------------------+---+---+
|   Empresa A|123456780001|                00|  0|  1|
|   Empresa B|987654320001|                12|   |   |
|   Empresa C|456789120001|                00|  0|  0|
|   Empresa D|321654980001|                34|   |   |
|   Empresa E|654123780001|                00|  0|  0|
|   Empresa F|789321450001|                00|  1|  0|
|   Empresa G|852963740001|                00|  0|  0|
|   Empresa H|963852140001|                78|   |   |
|   Empresa I|741852960001|                00|  0|  0|
|   Empresa J|258147360001|                90|   |   |
+------------+------------+------------------+---+---+



---